In [ ]:
## !/usr/bin/env python
#Config
from pyspark import SparkConf, SparkContext, HiveContext
#import re
import numpy as np
import pandas as pd
#import datetime
#from pyspark.mllib.regression import LabeledPoint
#from pyspark.mllib.feature import HashingTF
#from pyspark.mllib.classification import LogisticRegressionWithSGD, NaiveBayes, NaiveBayesModel
#import scipy.sparse as sps
#from pyspark.mllib.linalg import Vectors
#import sklearn
import itertools
import datetime
import sys

try:
    sc.stop()
except NameError:
    pass

conf = SparkConf().set("spark.executor.instances", 32).set("spark.driver.maxResultSize", "32g")
sc = SparkContext()
hc = HiveContext(sc)
sc.setCheckpointDir('checkpoint/')

In [ ]:
ymds = ['2016-10-18','2016-10-25','2016-11-01']

visit_r_query_pattern = '''
set hive.exec.dynamic.partition.mode=nonstrict;
insert overwrite
 table prod_features_liveinternet.visits_r partition(ymd)    
select id
     , urlfr
     , min(named_struct( 't', if((split(urlfr,'#')[0] <> ref_host) and not ref_host in ('','-','B'), time, time + 10000000)
     , 'ref_host', ref_host)).ref_host as ref_host
     , cast(count(*) as int) hits
     , cast(sum(hits_old) as int) hits_old
     , cast(count(distinct url) as int) as page_cnt
     , cast(MAX(time) - MIN(time) as int) as duration
     , AVG(time) as avg_time
     , stddev(time) as time_std
     , max(email_visit_flag) as email_visit
     , max(email_ru_flag) as emailru
     , max(utm_flag) as utm
     , max(if(regexp_extract(ref_host,'(.*)\\\\.[a-zA-Z]*$',1) in ('google','google.co','google.com','yandex','go.mail','nova.rambler','bing','yahoo'),1, 0)) as ref_search
     , max(if(regexp_extract(ref_host,'([^.]*)\\\\.[a-zA-Z]*$',1) in ('vk','facebook','ok','odnoklassniki','instagram','youtube'),1, 0)) as ref_social
     , max(ip) as ip
     , count(distinct ip) as ip_cnt
     , max(current_timestamp()) as load_dttm
     , ymd
  from(select id
            , time
            , concat(parse_url(concat('http://', url), 'HOST'), '#', path_fr) as urlfr
            , ymd
            , if(lower(concat(url,' ',referrer)) rlike '[^a-z](utm[_-]?[a-z]{0,15}|from|source|src(id)?)[%a-f0-9]{0,4}[=-_]?e?mail',1, 0) as email_visit_flag
            , if(lower(referrer) rlike 'e\\\\.mail\\\\.ru',1, 0) as email_ru_flag
            , if(lower(concat(url,' ',referrer)) rlike '([^a-zA-Z]utm(_|-|=|%[0-9][0-9a-fA-F]))|/?[yg]clid=',1, 0) as utm_flag
            , url
            , parse_url(concat('http://', referrer), 'HOST') as ref_host
            , hits_old
            , ip
         from(select id
                   , avg(cast(timestamp as Bigint)) as time
                   , url
                   , referrer
                   , ymd
                   , ip
                   , count(*) as hits_old
                from prod_raw_liveinternet.access_log
               where ymd = '#ymd'
               group
                  by id
                   , url
                   , referrer,ymd,ip,cast(timestamp/10 as Bigint)
            ) a
      LATERAL
         VIEW explode(split(parse_url(concat('http://', url), "PATH"), '/')) tt AS path_fr
     ) t
 group
    by ymd
     , id
     , urlfr
;

alter table prod_features_liveinternet.visits_r partition (ymd = '#ymd') concatenate

'''

pattern_test = '''
set hive.exec.dynamic.partition.mode=nonstrict;

insert overwrite
 table prod_features_liveinternet.visits_r partition(ymd)    
select id
     , urlfr
     , min(named_struct( 't', if((split(urlfr,'#')[0] <> ref_host) and not ref_host in ('','-','B'), time, time + 10000000)
     , 'ref_host', ref_host)).ref_host as ref_host
     , cast(count(*) as int) hits
     , cast(sum(hits_old) as int) hits_old
     , cast(count(distinct url) as int) as page_cnt
     , cast(MAX(time) - MIN(time) as int) as duration
     , AVG(time) as avg_time
     , stddev(time) as time_std
     , max(email_visit_flag) as email_visit
     , max(email_ru_flag) as emailru
     , max(utm_flag) as utm
     , max(if(regexp_extract(ref_host,'(.*)\\\\.[a-zA-Z]*$',1) in ('google','google.co','google.com','yandex','go.mail','nova.rambler','bing','yahoo'),1, 0)) as ref_search
     , max(if(regexp_extract(ref_host,'([^.]*)\\\\.[a-zA-Z]*$',1) in ('vk','facebook','ok','odnoklassniki','instagram','youtube'),1, 0)) as ref_social
     , max(ip) as ip
     , count(distinct ip) as ip_cnt
     , max(current_timestamp()) as load_dttm
     , ymd
  from(select id
            , time
            , concat(parse_url(concat('http://', url), 'HOST'), '#', path_fr) as urlfr
            , ymd
            , if(lower(concat(url,' ',referrer)) rlike '[^a-z](utm[_-]?[a-z]{0,15}|from|source|src(id)?)[%a-f0-9]{0,4}[=-_]?e?mail',1, 0) as email_visit_flag
            , if(lower(referrer) rlike 'e\\\\.mail\\\\.ru',1, 0) as email_ru_flag
            , if(lower(concat(url,' ',referrer)) rlike '([^a-zA-Z]utm(_|-|=|%[0-9][0-9a-fA-F]))|/?[yg]clid=',1, 0) as utm_flag
            , url
            , parse_url(concat('http://', referrer), 'HOST') as ref_host
            , hits_old
            , ip
         from(select id
                   , avg(cast(timestamp as Bigint)) as time
                   , url
                   , referrer
                   , ymd
                   , ip
                   , count(*) as hits_old
                from prod_raw_liveinternet.access_log
               where ymd = '#ymd'
                 and substr(id,1,5) = '00000'
               group
                  by id
                   , url
                   , referrer,ymd,ip,cast(timestamp/10 as Bigint)
            ) a
      LATERAL
         VIEW explode(split(parse_url(concat('http://', url), "PATH"), '/')) tt AS path_fr
     ) t
 group
    by ymd
 , id
     , urlfr
;

alter table prod_features_liveinternet.visits_r partition (ymd = '#ymd') concatenate

'''

for y in ymds:
    print(visit_r_query_pattern.replace('#ymd',y))
    #for q in pattern_test.replace('#ymd',y).split(';'): 
    #      hc.sql(q)

In [ ]:
signals_gen_query_pattern = '''
create table user_kposminin.la_many_features_#ind as
select
       v.ymd
     , b.contact_str as phone
     , max(if(p.id is Null,0,1)) as label
     , max(if(p.first_day = 1, 1, 0)) as first_day
     , max(score) as max_tcs_score
     , max(
         if(
          hour(v.avg_time) >= 9 and hour(v.avg_time) <= 20, 
          t.score,
          Null
         )
       ) as max_work_hours_tcs_score
     , avg(
         if(
           hour(v.avg_time) >= 9 and hour(v.avg_time) <= 20, 
           t.score,
           Null
         )    
       ) as avg_work_hours_tcs_score
     , count(distinct v.urlfr) as cnt
     , sum(v.hits) as hits
     , sum(v.ip_cnt) as ip_cnt
     , sum(v.page_cnt) as page_cnt
     , count(distinct v.id) as id_cnt
     , sum(if(hour(v.avg_time) >= 9 and hour(v.avg_time) <= 20,v.hits,0)) as work_hours_hits
     , min(hour(v.avg_time)) as min_avg_hour
     , max(hour(v.avg_time)) as max_avg_hour
     , avg(hour(v.avg_time)) as avg_avg_hour
     , sum(duration) as sum_duration
     , avg(duration) as avg_duration
     , sum(if(v.urlfr like 'e.mail.ru%',1,0)) as emailru
     , sum(if(v.urlfr like 'm.%',1,0))/sum(1) as mobile_share
     , sum(if(v.urlfr rlike '^(m\\.)?vk.com%', 1, 0))/sum(1) as vk_share
     , sum(if(v.urlfr like 'vk.com%' or v.urlfr rlike '^(m\\.)?ok\\.ru' or v.urlfr like 'm.odnoklassniki.ru%' or v.urlfr rlike '^(m\\.)?my.mail.ru',1,0))/sum(1) as social_share
     , stddev(v.avg_time) as hour_std     
     , log(avg((t.cnt_positive + 0.1)/(t.cnt_total - t.cnt_positive + 0.1))) as log_avg_exp_tsc_score
     , avg(t.score) as avg_tcs_score
     , sum(t.score) as sum_tcs_score
     , min(t.score) as min_tcs_score
     , percentile_approx(t.score,0.75) as q75_tcs_score
     , percentile_approx(t.score,0.25) as q25_tcs_score
     , percentile_approx(t.score,0.5) as q50_tcs_score
     , percentile_approx(t.score,0.9) as q90_tcs_score
     , count(distinct if(t.score > -7, v.urlfr,Null)) as good_tcs_urlfr_visited_cnt
     , count(distinct if(t.score > -5, v.urlfr,Null)) as very_good_tcs_urlfr_visited_cnt
     , max(if(v.ref_search = 1,t.score,Null)) as max_ref_search_score 
     , avg(if(v.ref_search = 1,t.score,Null)) as avg_ref_search_score 
     , percentile_approx(if(ref_search = 1,t.score,Null),0.9) as q90_ref_search_score 
     , sum(ref_search) as cnt_ref_search
     , max(
             named_struct( 
             'score',t.score, 
             'avg_hour',hour(v.avg_time)  
             )           
          ).avg_hour as max_urlfr_avg_hour
     , max(
             named_struct(
             'score', t.score,
             'hits', v.hits
             )           
          ).hits as max_urlfr_hits_cnt
     , max(
             named_struct(
             'score', t.score,
             'cnt_total', t.cnt_total
             )           
          ).cnt_total as max_urlfr_cnt_total

     , max(if(ref_social = 1,t.score,Null)) as max_ref_social_score 
     , avg(if(ref_social = 1,t.score,Null)) as avg_ref_social_score 
     , percentile_approx(if(ref_social = 1,t.score,Null),0.9) as q90_ref_social_score 
     , sum(ref_social) as cnt_ref_social

     , max(if(utm = 1,t.score,Null)) as max_utm_score 
     , avg(if(utm = 1,t.score,Null)) as avg_utm_score 
     , percentile_approx(if(utm = 1,t.score,Null),0.9) as q90_utm_score 
     , sum(utm) as cnt_utm
     
     , max(if(emailru = 1, t.score,Null)) as max_emailru_score 
     , avg(if(emailru = 1, t.score,Null)) as avg_emailru_score 
     , percentile_approx(if(emailru = 1, t.score,Null),0.9) as q90_emailru_score 
     , sum(emailru) as cnt_emailru

     , max(if(emailru + email_visit > 0, t.score,Null)) as max_email_score 
     , avg(if(emailru + email_visit > 0, t.score,Null)) as avg_email_score 
     , percentile_approx(if(emailru + email_visit > 0, t.score,Null), 0.9) as q90_email_score 
     , sum(if(emailru + email_visit > 0,1,0)) as cnt_email

  from prod_features_liveinternet.visits_r v
  left join (  
       select id, if(ymd = '#ymd1',1,0) as first_day from prod_features_liveinternet.user_action 
        where ymd between '#ymd1' and '#ymd3'
          and action_type = 'tinkoff_platinum_complete_application'
     ) p   on p.id = v.id
  left join (
       select urlfr
            , log((cnt_positive + 0.1)/(cnt_total - cnt_positive + 0.1)) as score
            , cnt_positive
            , cnt_total
         from prod_features_liveinternet.urlfr_tgt_cnt_cumulative2 a
        inner join 
             ( select max(ymd) as ymd
                 from prod_features_liveinternet.urlfr_tgt_cnt_cumulative2 
                where ymd < '#ymd0'
             ) b   on a.ymd = b.ymd
       where target  = 'tinkoff_platinum_complete_application@tinkoff_action'  
         and (cnt_total > 300000 or cnt_positive > 30)
     ) t on t.urlfr = v.urlfr
 inner join (select uid_str,h_uid_rk from prod_dds.h_uid where load_src = 'LI.02') d on d.uid_str = v.id
 inner join (select h_contact_rk, h_uid_rk from prod_dds.l_uid_contact where contact_type_cd = 'PHONE') c on d.h_uid_rk = c.h_uid_rk
 inner join (select h_contact_rk, contact_str from prod_dds.h_contact where contact_type_cd = 'PHONE') b on b.h_contact_rk = c.h_contact_rk  
 where 
       v.ymd = '#ymd0'
 group by 
       b.contact_str
     , v.ymd
;
       
'''

for y in ymds:
    yd = datetime.datetime.strptime(y, '%Y-%m-%d').date()
    signals_gen_query = signals_gen_query_pattern \
                               .replace('#ymd0',yd.strftime('%Y-%m-%d')) \
                               .replace('#ymd1',(yd + + datetime.timedelta(1)).strftime('%Y-%m-%d')) \
                               .replace('#ymd3',(yd + + datetime.timedelta(3)).strftime('%Y-%m-%d')) \
                               .replace('#ind',y.replace('-',''))                
    print(signals_gen_query)
    #for q in signals_gen_query: 
    #     hc.sql(q)

In [74]:
sample_query = '''
select * from user_kposminin.la_many_features_20161025 
where substr(md5(phone),1,1) = '0' or label = 1
'''
train = hc.sql(sample_query) \
    .toPandas()

In [77]:
test_sample = hc.sql('select * from user_kposminin.la_many_features_20161103 where substr(md5(phone),1,1) = "#k"'.replace('#k','4')) \
    .toPandas()

In [75]:
feat_cols = [u'max_tcs_score',
       u'max_work_hours_tcs_score', u'avg_work_hours_tcs_score', u'cnt',
       u'hits', u'ip_cnt', u'page_cnt', u'id_cnt', u'work_hours_hits',
       u'min_avg_hour', u'max_avg_hour', u'avg_avg_hour', u'sum_duration',
       u'avg_duration', u'emailru', u'mobile_share', u'vk_share',
       u'social_share', u'hour_std', u'log_avg_exp_tsc_score',
       u'avg_tcs_score', u'sum_tcs_score', u'min_tcs_score', u'q75_tcs_score',
       u'q25_tcs_score', u'q50_tcs_score', u'q90_tcs_score',
       u'good_tcs_urlfr_visited_cnt', u'very_good_tcs_urlfr_visited_cnt',
       u'max_ref_search_score', u'avg_ref_search_score',
       u'q90_ref_search_score', u'cnt_ref_search', u'max_urlfr_avg_hour',
       u'max_urlfr_hits_cnt', u'max_urlfr_cnt_total', u'max_ref_social_score',
       u'avg_ref_social_score', u'q90_ref_social_score', u'cnt_ref_social',
       u'max_utm_score', u'avg_utm_score', u'q90_utm_score', u'cnt_utm',
       u'max_emailru_score', u'avg_emailru_score', u'q90_emailru_score',
       u'cnt_emailru', u'max_email_score', u'avg_email_score',
       u'q90_email_score', u'cnt_email']
X = train[feat_cols]
y = train['label']

In [1]:
scaler = sklearn.preprocessing.StandardScaler()
imp = sklearn.preprocessing.Imputer(strategy='min')
X = scaler.fit_transform(imp.fit_transform(train[feat_cols]))
y = train['label']
Xt = scaler.transform(imp.transform(test_sample[feat_cols]))
yt = test_sample['label']
clfLogRegr = sklearn.linear_model.LogisticRegression(penalty = 'l1')

NameError: name 'sklearn' is not defined

In [2]:
train.count()

NameError: name 'train' is not defined

In [9]:
import sklearn
import xgboost as xgb

clfXGB = xgb.XGBClassifier(
 learning_rate = 0.1,
 n_estimators=100,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.7,
 colsample_bytree=0.7,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight = 1,
 seed=27
 )

clfXGB.fit(X,y)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.7,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=5,
       min_child_weight=1, missing=None, n_estimators=100, nthread=4,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.7)

In [17]:
print('{0} train AUCROC: {1}'.format('XGBoost',sklearn.metrics.roc_auc_score(
                y_true = y, 
                y_score = clfXGB.predict_proba(X)[:,1]
    )))

XGBoost train AUCROC: 0.840925617616


In [54]:
test = []
for k in list('0123456789abcdef'):
    batch = hc.sql('select * from user_kposminin.la_many_features_20161103 where substr(md5(phone),1,1) = "#k"'.replace('#k',k)) \
    .toPandas()
    batch['XGB pred'] = clfXGB.predict_proba(batch[feat_cols])[:,1]
    test = test + batch[['label','max_tcs_score','XGB pred']].values.tolist()

[0.0, -11.074343495190838, 0.0006523363990709186]

In [58]:
#test['XGB pred'] = clfXGB.predict_proba(test[feat_cols])[:,1]
print('{0} test XGB pred AUCROC: {1}'.format('XGBoost',sklearn.metrics.roc_auc_score(
                y_true = [e[0] for e in test], 
                y_score = [e[2] for e in test]
    )))

XGBoost test XGB pred AUCROC: 0.719881650568


In [59]:
print('{0} test max_tcs_score AUCROC: {1}'.format('XGBoost',sklearn.metrics.roc_auc_score(
                y_true = [e[0] for e in test],
                y_score = [e[1] for e in test]
    )))

XGBoost test max_tcs_score AUCROC: 0.736057171404


In [23]:
sorted(zip(clfXGB.feature_importances_,feat_cols),reverse=True)

[(0.057923496, u'max_tcs_score'),
 (0.04335155, u'max_ref_social_score'),
 (0.040801458, u'max_urlfr_cnt_total'),
 (0.039344262, u'avg_duration'),
 (0.034972679, u'avg_ref_search_score'),
 (0.032786883, u'max_work_hours_tcs_score'),
 (0.032058287, u'hour_std'),
 (0.031693988, u'q90_tcs_score'),
 (0.031693988, u'avg_email_score'),
 (0.030965392, u'avg_avg_hour'),
 (0.029143898, u'log_avg_exp_tsc_score'),
 (0.029143898, u'avg_work_hours_tcs_score'),
 (0.0287796, u'q25_tcs_score'),
 (0.026593806, u'social_share'),
 (0.026593806, u'q90_ref_social_score'),
 (0.026229508, u'max_ref_search_score'),
 (0.02550091, u'q90_ref_search_score'),
 (0.023679417, u'q50_tcs_score'),
 (0.023679417, u'avg_tcs_score'),
 (0.023315119, u'q75_tcs_score'),
 (0.022586521, u'min_avg_hour'),
 (0.022222223, u'avg_utm_score'),
 (0.021493625, u'sum_duration'),
 (0.021493625, u'max_urlfr_avg_hour'),
 (0.021493625, u'avg_ref_social_score'),
 (0.021129327, u'cnt'),
 (0.020036429, u'id_cnt'),
 (0.018943533, u'min_tcs_sco

In [60]:
#sl = sorted(zip(scores,labels),reverse = True)
sl = sorted([[e[2],e[0]] for e in test],reverse = True) # xgb

In [79]:
data = sl
def nvl(a,b):
    if a:
        return a
    else:
        return b
    
print('\nLift table for XGBoost\n\n{:<10}\t{:<10}\t{:<10}\t{:<10}\t{:<10}\t{:<10}\t{:<10}'.format(
        'q','pos_cnt','pos%','cnt','pos/cnt','lift','score'))
#pos_tot = sum([e[1] for e in data])
#pos_share_tot = float(pos_tot) / len(data)
for q in [0.9999,0.9998,0.9997,0.9995,0.999,0.998,0.997,0.995,0.99,0.98,0.95,0.9,0.8,0.7,0.5,0.3,0]:
    pos_cnt = sum([e[1] for e in data[:int((1-q)*len(data))]])
    cnt = int((1-q)*len(data))
    pos_share = float(pos_cnt)/cnt
    print('{:<10}\t{:<10}\t{:<10.2%}\t{:<10}\t{:<10.5%}\t{:<10.2f}\t{:<10.5f}'.format(q,
                                                                                      pos_cnt,
                                                                                      float(pos_cnt)/pos_tot,
                                                                                      cnt,pos_share,
                                                                                      pos_share/pos_share_tot,
                                                                                      nvl(data[int((1-q)*len(data))-1][0],-20)
                                                                                     ))


Lift table for XGBoost

q         	pos_cnt   	pos%      	cnt       	pos/cnt   	lift      	score     
0.9999    	33.0      	0.79%     	2322      	1.42119%  	78.98     	0.39762   
0.9998    	78.0      	1.87%     	4645      	1.67922%  	93.32     	0.31385   
0.9997    	122.0     	2.92%     	6968      	1.75086%  	97.30     	0.26137   
0.9995    	183.0     	4.38%     	11614     	1.57568%  	87.57     	0.19714   
0.999     	264.0     	6.32%     	23229     	1.13651%  	63.16     	0.11277   
0.998     	361.0     	8.64%     	46459     	0.77703%  	43.18     	0.05563   
0.997     	399.0     	9.55%     	69688     	0.57255%  	31.82     	0.03998   
0.995     	468.0     	11.20%    	116148    	0.40293%  	22.39     	0.02952   
0.99      	585.0     	14.00%    	232296    	0.25183%  	14.00     	0.02125   
0.98      	775.0     	18.54%    	464593    	0.16681%  	9.27      	0.01572   
0.95      	1144.0    	27.37%    	1161482   	0.09849%  	5.47      	0.01037   
0.9       	1712.0    	40.96%    	2322965   	0.07370

In [132]:
import math
sl1 = sorted([[e[1] if not math.isnan(e[1]) else -30,e[0]] for e in test],reverse = True)

In [133]:
data = sl1
def nvl(a,b):
    if a:
        return a
    else:
        return b
    
print('\nLift table for max score\n\n{:<10}\t{:<10}\t{:<10}\t{:<10}\t{:<10}\t{:<10}\t{:<10}'.format(
        'q','pos_cnt','pos%','cnt','pos/cnt','lift','score'))
pos_tot = sum([e[1] for e in data])
pos_share_tot = float(pos_tot) / len(data)
for q in [0.9999,0.9998,0.9997,0.9995,0.999,0.998,0.997,0.995,0.99,0.98,0.95,0.9,0.8,0.7,0.5,0.3,0]:
    pos_cnt = sum([e[1] for e in data[:int((1-q)*len(data))]])
    cnt = int((1-q)*len(data))
    pos_share = float(pos_cnt)/cnt
    print('{:<10}\t{:<10}\t{:<10.2%}\t{:<10}\t{:<10.5%}\t{:<10.2f}\t{:<10.5f}'.format(q,
                                                                                      pos_cnt,
                                                                                      float(pos_cnt)/pos_tot,
                                                                                      cnt,pos_share,
                                                                                      pos_share/pos_share_tot,
                                                                                      nvl(data[int((1-q)*len(data))-1][0],-20)
                                                                                     ))


Lift table for max score

q         	pos_cnt   	pos%      	cnt       	pos/cnt   	lift      	score     
0.9999    	47.0      	1.12%     	2322      	2.02412%  	112.49    	-2.83254  
0.9998    	64.0      	1.53%     	4645      	1.37783%  	76.57     	-3.14833  
0.9997    	84.0      	2.01%     	6968      	1.20551%  	66.99     	-3.19567  
0.9995    	144.0     	3.44%     	11614     	1.23988%  	68.90     	-3.45546  
0.999     	236.0     	5.65%     	23229     	1.01597%  	56.46     	-3.89131  
0.998     	373.0     	8.92%     	46459     	0.80286%  	44.62     	-4.32704  
0.997     	432.0     	10.33%    	69688     	0.61991%  	34.45     	-5.26854  
0.995     	918.0     	21.96%    	116148    	0.79037%  	43.92     	-6.00437  
0.99      	982.0     	23.49%    	232296    	0.42274%  	23.49     	-7.37057  
0.98      	1108.0    	26.51%    	464593    	0.23849%  	13.25     	-8.09208  
0.95      	1469.0    	35.14%    	1161482   	0.12648%  	7.03      	-8.54771  
0.9       	1819.0    	43.52%    	2322965   	0.078

In [72]:
#type(sl1[10002][0])
#data = sorted(sl1,key  = lambda e: -e[0])


NameError: name 'test' is not defined

In [68]:
pred = [float(v) for v in open('lgbm_files/LightGBM_predict_result.txt','r').read().split('\n')[:-1]]
labels = [int(v) for v in open('/data1/share/kosm/data/la_many_feat_labels_20161103_1.txt','r').read()]

In [65]:
! ls lgbm_files/

bigr_lgbm.model			  log1.txt	      nohup.out
bigr_model_trunc.model		  log_20161213.txt    readme.md
calc_auc.py			  log_20161214_2.txt  test.conf
ccall_many_feat_lgbm_trunc.model  log_20161214.txt    train.conf
lgbm.model			  log_short.txt       tst.txt
LightGBM_predict_result.txt	  log.txt


In [69]:
print('{0} test LGBM AUCROC: {1}'.format('XGBoost',sklearn.metrics.roc_auc_score(
                y_true = labels,
                y_score = pred
    )))

XGBoost test LGBM AUCROC: 0.76008302095


In [71]:
#data = sorted(zip(pred,labels),reverse = True)

def nvl(a,b):
    if a:
        return a
    else:
        return b
    
print('\nLift table for LGBM \n\n{:<10}\t{:<10}\t{:<10}\t{:<10}\t{:<10}\t{:<10}\t{:<10}'.format(
        'q','pos_cnt','pos%','cnt','pos/cnt','lift','score'))
#pos_tot = sum([e[1] for e in data])
#pos_share_tot = float(pos_tot) / len(data)
for q in [0.9999,0.9998,0.9997,0.9995,0.999,0.9985,0.9984,0.9983,0.9982,0.998,0.997,0.995,0.99,0.98,0.95,0.9,0.8,0.7,0.5,0.3,0]:
    pos_cnt = sum([e[1] for e in data[:int((1-q)*len(data))]])
    cnt = int((1-q)*len(data))
    pos_share = float(pos_cnt)/cnt
    print('{:<10}\t{:<10}\t{:<10.2%}\t{:<10}\t{:<10.5%}\t{:<10.2f}\t{:<10.5f}'.format(q,
                                                                                      pos_cnt,
                                                                                      float(pos_cnt)/pos_tot,
                                                                                      cnt,pos_share,
                                                                                      pos_share/pos_share_tot,
                                                                                      nvl(data[int((1-q)*len(data))-1][0],-20)
                                                                                     ))


Lift table for LGBM 

q         	pos_cnt   	pos%      	cnt       	pos/cnt   	lift      	score     
0.9999    	346       	8.28%     	2322      	14.90095% 	828.10    	0.91497   
0.9998    	346       	8.28%     	4645      	7.44887%  	413.96    	0.91497   
0.9997    	346       	8.28%     	6968      	4.96556%  	275.95    	0.91497   
0.9995    	346       	8.28%     	11614     	2.97916%  	165.56    	0.91497   
0.999     	346       	8.28%     	23229     	1.48952%  	82.78     	0.91497   
0.9985    	346       	8.28%     	34844     	0.99300%  	55.18     	0.91497   
0.9984    	346       	8.28%     	37167     	0.93093%  	51.74     	0.91497   
0.9983    	363       	8.68%     	39490     	0.91922%  	51.08     	0.91488   
0.9982    	386       	9.23%     	41813     	0.92316%  	51.30     	0.91260   
0.998     	390       	9.33%     	46459     	0.83945%  	46.65     	0.91216   
0.997     	432       	10.33%    	69688     	0.61991%  	34.45     	0.81949   
0.995     	484       	11.58%    	116148    	0.41671% 

In [5]:
feat_cols = [u'max_tcs_score',
       u'max_work_hours_tcs_score', u'avg_work_hours_tcs_score', u'cnt',
       u'hits', u'ip_cnt', u'page_cnt', u'id_cnt', u'work_hours_hits',
       u'min_avg_hour', u'max_avg_hour', u'avg_avg_hour', u'sum_duration',
       u'avg_duration', u'emailru', u'mobile_share', u'vk_share',
       u'social_share', u'hour_std', u'log_avg_exp_tsc_score',
       u'avg_tcs_score', u'sum_tcs_score', u'min_tcs_score', u'q75_tcs_score',
       u'q25_tcs_score', u'q50_tcs_score', u'q90_tcs_score',
       u'good_tcs_urlfr_visited_cnt', u'very_good_tcs_urlfr_visited_cnt',
       u'max_ref_search_score', u'avg_ref_search_score',
       u'q90_ref_search_score', u'cnt_ref_search', u'max_urlfr_avg_hour',
       u'max_urlfr_hits_cnt', u'max_urlfr_cnt_total', u'max_ref_social_score',
       u'avg_ref_social_score', u'q90_ref_social_score', u'cnt_ref_social',
       u'max_utm_score', u'avg_utm_score', u'q90_utm_score', u'cnt_utm',
       u'max_emailru_score', u'avg_emailru_score', u'q90_emailru_score',
       u'cnt_emailru', u'max_email_score', u'avg_email_score',
       u'q90_email_score', u'cnt_email']
feat_import = [[11,283],
[13,265],
[18,257],
[0,238],
[35,238],
[17,222],
[22,216],
[3,207],
[31,186],
[24,176],
[25,175],
[29,172],
[21,169],
[8,165],
[23,165],
[9,160],
[30,153],
[4,153],
[26,152],
[19,152],
[12,151],
[33,149],
[2,140],
[1,131],
[20,130],
[36,124],
[37,120],
[34,118],
[39,116],
[32,111],
[6,111],
[5,111],
[38,110],
[7,84],
[10,77],
[28,61],
[43,59],
[15,48],
[49,48],
[40,44],
[41,40],
[42,31],
[14,30],
[50,29],
[47,25],
[51,24],
[48,23],
[45,18],
[27,15],
[46,11],
[44,7]
]

In [6]:
for i,v in feat_import:
    print(feat_cols[i],v)

(u'avg_avg_hour', 283)
(u'avg_duration', 265)
(u'hour_std', 257)
(u'max_tcs_score', 238)
(u'max_urlfr_cnt_total', 238)
(u'social_share', 222)
(u'min_tcs_score', 216)
(u'cnt', 207)
(u'q90_ref_search_score', 186)
(u'q25_tcs_score', 176)
(u'q50_tcs_score', 175)
(u'max_ref_search_score', 172)
(u'sum_tcs_score', 169)
(u'work_hours_hits', 165)
(u'q75_tcs_score', 165)
(u'min_avg_hour', 160)
(u'avg_ref_search_score', 153)
(u'hits', 153)
(u'q90_tcs_score', 152)
(u'log_avg_exp_tsc_score', 152)
(u'sum_duration', 151)
(u'max_urlfr_avg_hour', 149)
(u'avg_work_hours_tcs_score', 140)
(u'max_work_hours_tcs_score', 131)
(u'avg_tcs_score', 130)
(u'max_ref_social_score', 124)
(u'avg_ref_social_score', 120)
(u'max_urlfr_hits_cnt', 118)
(u'cnt_ref_social', 116)
(u'cnt_ref_search', 111)
(u'page_cnt', 111)
(u'ip_cnt', 111)
(u'q90_ref_social_score', 110)
(u'id_cnt', 84)
(u'max_avg_hour', 77)
(u'very_good_tcs_urlfr_visited_cnt', 61)
(u'cnt_utm', 59)
(u'mobile_share', 48)
(u'avg_email_score', 48)
(u'max_utm_sco

In [58]:
import sklearn
print('{0} test max_tcs_score AUCROC: {1}'.format('XGBoost',sklearn.metrics.roc_auc_score(
                y_true = train['label'],
                y_score = train[u'page_cnt']
    )))

XGBoost test max_tcs_score AUCROC: 0.654969600083


In [24]:
train.count()

ymd                                1516397
phone                              1516397
label                              1516397
first_day                          1516397
max_tcs_score                      1488162
max_work_hours_tcs_score           1343724
avg_work_hours_tcs_score           1343724
cnt                                1516397
hits                               1516397
ip_cnt                             1516397
page_cnt                           1516397
id_cnt                             1516397
work_hours_hits                    1516397
min_avg_hour                       1516397
max_avg_hour                       1516397
avg_avg_hour                       1516397
sum_duration                       1516397
avg_duration                       1516397
emailru                            1516397
mobile_share                       1516397
vk_share                           1516397
social_share                       1516397
hour_std                           1516397
log_avg_exp

In [1]:
import datetime

In [9]:
a = '2016-10-13'
(datetime.datetime.strptime(a,'%Y-%m-%d').date() + datetime.timedelta(days = -3)).strftime('%Y-%m-%d')

'2016-10-10'